Here we use a scenario where we first generate the outline for the given blog topic after that considering that outline we are genarting the whole blog. And this lies under prompt chaining because here there are two nodes and both are interacting with LLMs.

In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [9]:
load_dotenv()

model = ChatGroq(model="llama-3.1-8b-instant")

In [10]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [11]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [12]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [13]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [15]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)
print(final_state['title'])
print(final_state['outline'])
print(final_state['content'])

{'title': 'Rise of AI in India', 'outline': '**Title:** "The Rise of AI in India: Unlocking Opportunities and Challenges"\n\n**I. Introduction**\n\n* Brief overview of Artificial Intelligence (AI) and its growth globally\n* Importance of AI in India\'s economy and societal development\n* Thesis statement: India is poised to become a major hub for AI innovations, driven by government initiatives, innovative startups, and growing demand for AI-powered solutions.\n\n**II. Government Initiatives and Policies**\n\n* Overview of the Indian government\'s vision for AI, as outlined in the National Policy on Artificial Intelligence (2019)\n* Details on the establishment of the National Artificial Intelligence Portal and the AI Ethics Committee\n* Analysis of the government\'s efforts to promote AI research, development, and deployment in various sectors (e.g., healthcare, education, finance)\n* Examples of government-funded AI projects and initiatives (e.g., AI for Social Good, AI for Agricultu